## Inventory Simulation Code (90 Days, All Products)

In [2]:
import numpy as np
import pandas as pd

In [3]:
# Load the rules and original dataset
rules = pd.read_csv('inventory_rules.csv')
sales = pd.read_csv('Sales_Data.csv')

In [4]:
# Merge to get initial inventory, price, and lead time
products = sales.groupby('product_ID').agg({
    'inventory_level': 'last',
    'price': 'mean',
    'order_lead_time': 'mean',
    'demand_quantity': ['mean', 'std']
}).reset_index()
products.columns = ['product_ID', 'inventory_level', 'price', 'lead_time', 'demand_mean', 'demand_std']


In [5]:
# Merge with rules
inventory = pd.merge(products, rules, on='product_ID')


In [6]:
# Simulation settings
simulation_days = 90
Z = 1.65
ordering_cost = 200
holding_cost_rate = 0.15
results = []


In [8]:
# Drop rows where any of the required fields are missing
required_columns = ['inventory_level', 'lead_time', 'demand_mean', 'demand_std', 'ROP', 'EOQ', 'price']
clean_inventory = inventory.dropna(subset=required_columns)

# Optional: Also filter rows with invalid values (e.g., negative or zero)
clean_inventory = clean_inventory[
    (clean_inventory['lead_time'] > 0) &
    (clean_inventory['EOQ'] > 0) &
    (clean_inventory['demand_mean'] > 0) &
    (clean_inventory['price'] > 0)
].copy()

In [9]:
# Run simulation per product
for _, row in clean_inventory.iterrows():
    inv = row['inventory_level']
    on_order = 0
    lead_time = int(row['lead_time'])
    pipeline = [0] * lead_time
    daily_demand_mu = row['demand_mean']
    daily_demand_sigma = row['demand_std']
    reorder_point = row['ROP']
    order_qty = int(row['EOQ'])
    price = row['price']
    holding_cost_per_unit = price * holding_cost_rate / 365

    stockouts = 0
    total_orders = 0
    holding_cost = 0
    total_demand = 0
    fulfilled_demand = 0

    for day in range(simulation_days):
        # Receive order if any
        inv += pipeline.pop(0)
        pipeline.append(0)

        # Demand today
        demand = max(0, int(np.random.normal(daily_demand_mu, daily_demand_sigma)))
        total_demand += demand

        if inv >= demand:
            fulfilled_demand += demand
            inv -= demand
        else:
            fulfilled_demand += inv
            inv = 0
            stockouts += 1

        # Reorder decision
        if inv + sum(pipeline) <= reorder_point:
            pipeline[-1] += order_qty
            total_orders += 1

        # Holding cost
        holding_cost += inv * holding_cost_per_unit

    service_level = fulfilled_demand / total_demand if total_demand else 0

    results.append({
        'product_ID': row['product_ID'],
        'product_name': row['product_name'],
        'Total_Orders': total_orders,
        'Stockout_Days': stockouts,
        'Service_Level': round(service_level, 2),
        'Avg_Holding_Cost': round(holding_cost, 2)
    })



In [11]:
# Create simulation report
simulation_report = pd.DataFrame(results)
simulation_report.to_csv('inventory_simulation_report.csv', index=False)
simulation_report.head()

,product_ID,product_name,Total_Orders,Stockout_Days,Service_Level,Avg_Holding_Cost
0,10114,Stainless Steel Straws,90,69,0.01,3874.16
1,10335,Recipe Book Stand,90,81,0.02,1510.12
2,10447,Digital Kitchen Timer,90,75,0.01,2348.51
3,13101,Pumpkin Waffles,90,88,0.01,249.84
4,15006,Luxury Bathrobe,90,69,0.01,2137.76
